In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Aljazeera web scrapping

In this data scraping process, the goal was to extract information from the Al Jazeera website. The first step involved scraping the "Category" section to gather the URLs corresponding to different categories. These URLs were then used to navigate to each category's page and extract the URLs of individual articles along with their titles. After obtaining the article URLs and titles, the next step was to extract the text content from each article. Finally, all the scraped data was consolidated into a pandas DataFrame with six columns: "id" (unique identifier), "url" (article URL), "titre" (article title), "categorie" (category), "ArabCategory" (optional Arabic category), and "texte" (article text). The DataFrame consists of 350 entries and provides a comprehensive collection of data for further analysis or processing.

In [2]:
import requests
from bs4 import BeautifulSoup
import csv

# Make a GET request to the URL of the page you want to scrape
url = "https://www.aljazeera.net/"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
# Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
# Find the ul tag containing the items
ul_element = soup.find("ul", class_="menu header-menu")

categories = []
links = []

# Iterate over the items in the ul tag to extract the categories and links
for li_element in ul_element.find_all("li", class_="menu__item"):
    category = li_element.find("span").text.strip()
    link = li_element.find("a")["href"]
    categories.append(category)
    links.append(link)

# Create a list of dictionaries
data = []
for category, link in zip(categories, links):
    data.append({"Category": category, "Link": link})

# Write the data to a CSV file
with open("categories.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["Category", "Link"])
    writer.writeheader()  # Write the CSV header
    writer.writerows(data)

print("Extraction completed. The data has been saved in 'categories.csv'.")

Extraction completed. The data has been saved in 'categories.csv'.


In [3]:
# Load the data/kaggle/working/categories2.csv
df = pd.read_csv("/kaggle/working/categories.csv", nrows=100)
df.head()

,Category,Link
0,أخبار,/news/
1,طوفان الأقصى,/where/palestine/
2,اقتصاد,/ebusiness/
3,ثقافة,/culture/
4,رياضة,/sport/


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  25 non-null     object
 1   Link      25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes


In [5]:
import requests
from bs4 import BeautifulSoup
import csv

# Make a GET request to the URL of the page you want to scrape
url = "https://www.aljazeera.net/"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the ul tag containing the links
    ul_element = soup.find("ul", class_="menu header-menu")

    # Extract the URLs
    urls = []
    for li_element in ul_element.find_all("li", class_="menu__item"):
        link = "https://www.aljazeera.net/" + li_element.find("a")["href"]
        urls.append(link)

# Créer une liste pour stocker les informations des articles
articles_data = []
article_id = 0

# Parcourir chaque URL
for url in urls:
    # Envoyer une requête GET à l'URL
    response = requests.get(url)

    # Vérifier que la requête s'est bien déroulée
    if response.status_code == 200:
        # Utiliser Beautiful Soup pour analyser le contenu HTML de la page
        soup = BeautifulSoup(response.content, "html.parser")

        # Trouver tous les éléments <a> contenant les liens des articles
        article_links = soup.find_all("a", class_="u-clickable-card__link")

        # Extraire les informations des articles
        for link in article_links:
            article_id += 1
            article_url = "https://www.aljazeera.net" + link["href"]
            article_title = link.span.get_text()

            # Extraire la catégorie de l'URL
            category = url.split("/")[-2]

            # Ajouter les informations dans la liste
            articles_data.append({"id": article_id, "url": article_url, "titre": article_title, "categorie": category})

# Écrire les données dans un fichier CSV
with open("articles.csv", "w", encoding="utf-8", newline="") as csvfile:
    fieldnames = ["id", "url", "titre", "categorie"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Écrire les en-têtes du fichier CSV
    writer.writeheader()

    # Écrire les données des articles
    writer.writerows(articles_data)

print("Le fichier CSV a été généré avec succès.")

Le fichier CSV a été généré avec succès.


In [6]:
# Load the data
df = pd.read_csv("/kaggle/working/articles.csv")
df.head()


,id,url,titre,categorie
0,1,https://www.aljazeera.net/news/2023/12/28/%d9%...,بن شبات: الوضع صار أفضل لحماس وأصعب على قواتنا,news
1,2,https://www.aljazeera.net/news/2023/12/28/%d8%...,صباح أطفال غزة,news
2,3,https://www.aljazeera.net/news/2023/12/28/%d8%...,حميدتي في أديس أبابا بعد تعذر ذهابه للقاء البرهان,news
3,4,https://www.aljazeera.net/news/2023/12/28/%d8%...,صحافة عالمية: نتنياهو يطيل الحرب حفاظا على منص...,news
4,5,https://www.aljazeera.net/news/2023/12/28/%d8%...,الجزيرة تنقل معاناة النازحين للحصول على الدقيق...,news


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         322 non-null    int64 
 1   url        322 non-null    object
 2   titre      322 non-null    object
 3   categorie  322 non-null    object
dtypes: int64(1), object(3)
memory usage: 10.2+ KB


In [1]:
import requests
from bs4 import BeautifulSoup
import csv

# Make a GET request to the URL of the page you want to scrape
url = "https://www.aljazeera.net/"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the ul tag containing the links
    ul_element = soup.find("ul", class_="menu header-menu")

    # Extract the URLs
    urls = []
    for li_element in ul_element.find_all("li", class_="menu__item"):
        link = "https://www.aljazeera.net/" + li_element.find("a")["href"]
        urls.append(link)

    # Create a list to store the article information
    articles_data = []
    article_id = 0

    # Iterate over each URL
    for url in urls:
        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Use Beautiful Soup to parse the HTML content of the page
            soup = BeautifulSoup(response.content, "html.parser")

            # Find all the <a> elements containing the article links
            article_links = soup.find_all("a", class_="u-clickable-card__link")

            # Extract the article information
            for link in article_links:
                article_id += 1
                article_url = "https://www.aljazeera.net" + link["href"]
                article_title = link.span.get_text()

                # Extract the article summary from the HTML content
                summary = link.find_next("div", class_="gc__excerpt").p.get_text()

                # Extract the category from the URL
                category = url.split("/")[-2]

                # Add the information to the list
                articles_data.append({"id": article_id, "url": article_url, "title": article_title, "category": category, "summary": summary})

    # Write the data to a CSV file
    with open("articles.csv", "w", encoding="utf-8", newline="") as csvfile:
        fieldnames = ["id", "url", "title", "category", "summary"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the CSV file headers
        writer.writeheader()

        # Write the article data
        writer.writerows(articles_data)

    print("The CSV file has been generated successfully.")

The CSV file has been generated successfully.


In [9]:
# Load the data
df = pd.read_csv("/kaggle/working/articles.csv")
df.head()

,id,url,title,category,summary
0,1,https://www.aljazeera.net/news/2023/12/28/%d9%...,بن شبات: الوضع صار أفضل لحماس وأصعب على قواتنا,news,نقلت صحيفة وول ستريت جورنال عن مستشار الأمن ال...
1,2,https://www.aljazeera.net/news/2023/12/28/%d8%...,صباح أطفال غزة,news,مشاهد وصور تحكي واقع صباح أطفال غزة منذ السابع...
2,3,https://www.aljazeera.net/news/2023/12/28/%d8%...,حميدتي في أديس أبابا بعد تعذر ذهابه للقاء البرهان,news,وصل قائد قوات الدعم السريع السودانية الفريق أو...
3,4,https://www.aljazeera.net/news/2023/12/28/%d8%...,صحافة عالمية: نتنياهو يطيل الحرب حفاظا على منص...,news,تناولت العديد من الصحف الغربية تطورات الحرب ال...
4,5,https://www.aljazeera.net/news/2023/12/28/%d8%...,الجزيرة تنقل معاناة النازحين للحصول على الدقيق...,news,نقلت الجزيرة جانبا من معاناة سكان قطاع غزة من ...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        322 non-null    int64 
 1   url       322 non-null    object
 2   title     322 non-null    object
 3   category  322 non-null    object
 4   summary   322 non-null    object
dtypes: int64(1), object(4)
memory usage: 12.7+ KB


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Charger le DataFrame contenant les URLs
df = pd.read_csv("/kaggle/working/articles.csv")

# Parcourir chaque ligne du DataFrame
for index, row in df.iterrows():
    url = row['url']
    response = requests.get(url)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Analyser le contenu HTML avec BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Extraire le texte des balises <p>
        paragraphs = soup.find_all("p")

        # Concaténer le texte extrait des balises <p>
        text = ' '.join(p.get_text() for p in paragraphs)

        # Mettre à jour la colonne "texte" du DataFrame avec le texte extrait
        df.at[index, 'texte'] = text

    else:
        print(f"La requête a échoué pour l'URL {url}")

# Enregistrer le DataFrame mis à jour avec le texte extrait
df.to_csv('Aljazeera_dataset.csv', index=False)

In [33]:

# Charger le DataFrame contenant les URLs
df = pd.read_csv("/kaggle/working/Aljazeera_dataset.csv")
df.head(50)

,id,url,title,category,summary,texte
0,1,https://www.aljazeera.net/news/2023/12/28/%d9%...,بن شبات: الوضع صار أفضل لحماس وأصعب على قواتنا,news,نقلت صحيفة وول ستريت جورنال عن مستشار الأمن ال...,نقلت صحيفة وول ستريت جورنال عن مستشار الأمن ال...
1,2,https://www.aljazeera.net/news/2023/12/28/%d8%...,صباح أطفال غزة,news,مشاهد وصور تحكي واقع صباح أطفال غزة منذ السابع...,أول ما يخطر ببالنا مع إشراقة الشمس، هو الصباح ...
2,3,https://www.aljazeera.net/news/2023/12/28/%d8%...,حميدتي في أديس أبابا بعد تعذر ذهابه للقاء البرهان,news,وصل قائد قوات الدعم السريع السودانية الفريق أو...,وصل قائد قوات الدعم السريع السودانية الفريق أو...
3,4,https://www.aljazeera.net/news/2023/12/28/%d8%...,صحافة عالمية: نتنياهو يطيل الحرب حفاظا على منص...,news,تناولت العديد من الصحف الغربية تطورات الحرب ال...,تناولت العديد من الصحف الغربية تطورات الحرب ال...
4,5,https://www.aljazeera.net/news/2023/12/28/%d8%...,الجزيرة تنقل معاناة النازحين للحصول على الدقيق...,news,نقلت الجزيرة جانبا من معاناة سكان قطاع غزة من ...,نقلت الجزيرة جانبا من معاناة سكان قطاع غزة من ...
5,6,https://www.aljazeera.net/news/2023/12/28/3-%d...,3 شهداء بالضفة والاحتلال يصادر أموالا من شركات...,news,ارتفع عدد الشهداء في الضفة الغربية إلى 314 شهي...,ارتفع عدد الشهداء في الضفة الغربية إلى 314 شهي...
6,7,https://www.aljazeera.net/politics/2023/12/28/...,لوفيغارو: باريس تشكك في النوايا الحقيقية لواشن...,news,قالت لوفيغارو إن باريس متشككة في “أغراض” العمل...,"قالت صحيفة لوفيغارو إن باريس متشككة في ""أغراض""..."
7,8,https://www.aljazeera.net/news/2023/12/28/%d8%...,إسرائيل تعترف بمقتل أكثر من 500 ضابط وجندي منذ...,news,أعلن الجيش الإسرائيلي صباح اليوم الخميس مقتل ض...,أعلن الجيش الإسرائيلي صباح اليوم الخميس مقتل ض...
8,9,https://www.aljazeera.net/politics/2023/12/28/...,تقديس القتل العشوائي بغزة.. هآرتس: هزيمة ثانية...,news,قالت هآرتس إن معسكر اليمين الاستيطاني المسيحان...,قال تقرير لصحيفة هآرتس الإسرائيلية إن معسكر ال...
9,10,https://www.aljazeera.net/news/2023/12/28/%d9%...,لافروف: لا بد من رفع الظلم الذي عانى منه الفلس...,news,أكد وزير الخارجية الروسي سيرغي لافروف ضرورة كس...,أكد وزير الخارجية الروسي سيرغي لافروف ضرورة كس...


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        322 non-null    int64 
 1   url       322 non-null    object
 2   title     322 non-null    object
 3   category  322 non-null    object
 4   summary   322 non-null    object
 5   texte     322 non-null    object
dtypes: int64(1), object(5)
memory usage: 15.2+ KB


In [18]:
text1 = df.iloc[0][-1]
print(text1)

نقلت صحيفة وول ستريت جورنال عن مستشار الأمن القومي الإسرائيلي السابق مئير بن شبات أن حركة المقاومة الإسلامية (حماس) باتت تشعر بالثقة الكافية لرفض أي صفقة لا تحقق لها النصر بشكل مباشر في هذه الحرب المستمرة منذ السابع من أكتوبر/تشرين الأول الماضي. جاء ذلك في مقال لهيئة تحرير الصحيفة -نشر أمس الأربعاء تحت عنوان "لهذا لن تفرج حماس عن الرهائن"- حيث تساءلت فيه عن التغيرات التي حدثت منذ هدنة أواخر نوفمبر/تشرين الثاني الماضي وجعلت الحركة ترفض عرضين للتهدئة وتبادل الأسرى، وفقا للصحيفة، في إشارة إلى موقف حماس التي تصر حاليا على عدم التفاوض إلا بعد وقف إطلاق النار بشكل كامل. وقال بن شبات -الذي كان مستشارا للأمن القومي الإسرائيلي من 2017 إلى 2021- إن ثقة حماس ربما تكون على غير هدى، لكنها ليست بلا أساس. وأوضح أنه "بينما الظروف التي تعمل فيها قواتنا أصبحت أصعب مما كانت، فقد تحسنت الأمور لمقاتلي حماس". وذكرت الصحيفة أنه في ظل ضغوط إدارة الرئيس الأميركي جو بايدن صارت إسرائيل تستخدم "قوة نارية أقل" للتحضير لتوغلاتها البرية في غزة، وهذا يعطي حماس مزيدا من الفرص للإيقاع بالجنود الإسرائيليين في الكمائن.  

In [19]:
summ1 = df.iloc[0][-2]
print(summ1)

نقلت صحيفة وول ستريت جورنال عن مستشار الأمن القومي الإسرائيلي السابق مئير بن شبات أن حركة المقاومة الإسلامية (حماس) باتت تشعر بالثقة الكافية لرفض أي صفقة لا تحقق لها النصر بشكل مباشر.
